In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets ,models , transforms
import json
from torch.utils.data import Dataset, DataLoader ,random_split
from PIL import Image
from pathlib import Path
classLabels = ["desert", "mountains", "sea", "sunset", "trees" ]
print(torch.__version__)
from utils.datasets import MyDataset
from train import train

1.8.1+cu111


In [3]:
batch_size=32
transform = transforms.Compose([transforms.Resize((224,224)) , 
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                               ])

dataset = MyDataset("dataview/Labels.json" , Path("dataview/view1"), Path("dataview/view2") , transform)
valid_no = int(len(dataset)*0.12) 
trainset ,valset  = random_split( dataset , [len(dataset) -valid_no  ,valid_no])
print(f"trainset len {len(trainset)} valset len {len(valset)}")
dataloader = {"train":DataLoader(trainset , shuffle=True , batch_size=batch_size),
              "val": DataLoader(valset , shuffle=True , batch_size=batch_size)}

trainset len 120 valset len 16


# Model definition


In [4]:
from models.mlmodel import Volume
model = Volume(num_classes=136)

# Optimizer and Criterion

In [5]:
import torch.optim as optim
from torch.optim import lr_scheduler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cuda:1')
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()

# specify optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
sgdr_partial = lr_scheduler.CosineAnnealingLR(optimizer, T_max=5, eta_min=0.005 )

# Training

In [7]:
train(model,dataloader , criterion, optimizer,sgdr_partial,num_epochs=50)

Epochs:   0%|          | 0/50 [00:00<?, ?it/s]/home/le.minh.chien/miniconda3/envs/gpu_env/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Epochs:   2%|▏         | 1/50 [00:07<06:22,  7.80s/it]

['train Loss: 0.7249 Acc: 0.4786', 'val Loss: 2.8873 Acc: 0.5049']


Epochs:   4%|▍         | 2/50 [00:14<05:55,  7.40s/it]

['train Loss: 0.5538 Acc: 0.5642', 'val Loss: 152.1575 Acc: 0.5508']


/home/le.minh.chien/miniconda3/envs/gpu_env/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
Epochs:   6%|▌         | 3/50 [00:21<05:29,  7.01s/it]

...

['train Loss: 0.0837 Acc: 0.9403', 'val Loss: 0.0537 Acc: 0.9570']


Epochs:  98%|█████████▊| 49/50 [05:35<00:07,  7.01s/it]

['train Loss: 0.1027 Acc: 0.9407', 'val Loss: 0.0591 Acc: 0.9480']


Epochs: 100%|██████████| 50/50 [05:42<00:00,  6.85s/it]

['train Loss: 0.0673 Acc: 0.9507', 'val Loss: 0.0647 Acc: 0.9391']


In [8]:
def createCheckpoint(filename=Path("Volume_model.pt")):
  checkpoint = {
              'epoch': 5,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              "batch_size":batch_size,
  } # save all important stuff
  torch.save(checkpoint , filename)
createCheckpoint()

# test inference

In [ ]:
model

In [35]:
import os
import time

image = Image.open(os.path.join("dataview/view1", "20.png")).convert("RGB")
image_2 = Image.open(os.path.join("dataview/view2", "20.png")).convert("RGB")
image = transform(image)
image_2 = transform(image_2)
image = torch.unsqueeze(image, 0).to(device)
image_2 = torch.unsqueeze(image_2, 0).to(device)

start = time.time()
results = model(image, image_2)
print("Time to inference: ", time.time() - start)
results = torch.sigmoid(results)
print(results>0.7)

Time to inference:  0.009053468704223633
tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         Fa

In [27]:
image, image2, label = next(iter(dataloader["val"]))
image = image.to(device)
image2 = image2.to(device)

label = label.to(device)
output = 0
with torch.no_grad():
  output = model(image, image2)
  output = torch.sigmoid(output)
output = output>0.5
print(output)

tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False]], device='cuda:0')
